# Classificação com embeddings: BERTimbau

Este notebook implementa um modelo baseado em embeddings contextualizados
(BERTimbau) para a tarefa de classificação automática de tipos de resposta.

O objetivo é comparar o desempenho de um modelo neural pré-treinado com o
baseline clássico baseado em TF-IDF e Regressão Logística.


## Bibliotecas

Utilizamos o modelo BERTimbau pré-treinado para o português brasileiro,
disponibilizado pela comunidade HuggingFace.


In [2]:
!pip install -q transformers datasets accelerate


In [3]:
import pandas as pd
import numpy as np
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

from sklearn.metrics import accuracy_score, f1_score


## Carregamento do corpus

Utilizamos o mesmo corpus de modelagem empregado no baseline, garantindo
comparabilidade direta entre os modelos.


In [4]:
df = pd.read_csv("corpus_modelagem.csv")

df.shape


(239, 6)

## Codificação das classes

As categorias textuais são convertidas em rótulos numéricos para o treinamento
do modelo neural.


In [7]:
label2id = {label: i for i, label in enumerate(df["tipo_resposta"].unique())}
id2label = {i: label for label, i in label2id.items()}

df["label"] = df["tipo_resposta"].map(label2id)

label2id


{'subinformativa': 0, 'sobreinformativa': 1, 'completa': 2}

## Divisão treino/teste

Utilizamos a mesma proporção treino/teste adotada no baseline (80/20).


In [8]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

train_df.shape, test_df.shape


((191, 7), (48, 7))

## Conversão para o formato Dataset


In [9]:
train_dataset = Dataset.from_pandas(
    train_df[["resposta", "label"]]
)

test_dataset = Dataset.from_pandas(
    test_df[["resposta", "label"]]
)


## Tokenização com BERTimbau


In [10]:
model_name = "neuralmind/bert-base-portuguese-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["resposta"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

C:\Users\alice\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alice\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/191 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

## Definição do modelo

Utilizamos fine-tuning completo do BERTimbau com uma camada de classificação
linear.


In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: neuralmind/bert-base-portuguese-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. C

## Métricas

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }


## Configuração do treinamento


In [21]:
training_args = TrainingArguments(
    output_dir="bertimbau",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    report_to="none"
)



## Treinamento do modelo


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


## Avaliação final


In [26]:
results = trainer.evaluate()
results


C:\Users\alice\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


{'eval_loss': 1.1000031232833862,
 'eval_model_preparation_time': 0.0067,
 'eval_accuracy': 0.3958333333333333,
 'eval_f1_macro': 0.19191919191919193,
 'eval_runtime': 14.874,
 'eval_samples_per_second': 3.227,
 'eval_steps_per_second': 0.202}

## Comparação com o baseline

O modelo baseline baseado em TF-IDF e Regressão Logística apresentou desempenho
superior (acurácia ≈ 96%) em comparação ao modelo baseado em embeddings
contextualizados (BERTimbau), cujo desempenho foi consideravelmente inferior
(acurácia ≈ 40%).


In [27]:
import pandas as pd

resultados = pd.DataFrame({
    "Modelo": ["TF-IDF + LR", "BERTimbau"],
    "Representação": ["TF-IDF", "Embeddings contextualizados"],
    "Acurácia": [0.9583, 0.3958],
    "F1-macro": [0.95, 0.1919]
})

resultados


,Modelo,Representação,Acurácia,F1-macro
0,TF-IDF + LR,TF-IDF,0.9583,0.9500
1,BERTimbau,Embeddings contextualizados,0.3958,0.1919


In [28]:
resultados.to_csv("tabela_comparativa.csv", index=False)
